# Muon Identification

Muon identification with the ePIC detector is a hot topic right now. Muons are produced in a range of key interaction channels under investigation including time-like compton scattering (TCS), Vector Meson Production (J/$\psi$ -> $\mu$$\mu$) and beyond standard model searches. The challenge is that ePIC does **NOT** have dedicated muon detection systems.

Without dedicated systems, we must identify muons from charged tracks and signals in the calorimeters and PID detectors that we do have available. The difficulty is that muons are very similar in mass to pions, so misidentification of one as the other is a big issue.

- $m_{\pi} = 139.57039 MeV/c^{2}$
- $m_{\mu} = 105.66 MeV/c^{2}$

OK, not exactly the same, but quite close. Enough that with the detector resolution we have in HEP systems, it's an issue.

To try to reliably identify muons, we'll need to dive into calorimeter/PID/tracking systems and see if we can distinguish muon signals from other particles. Once we do this for "simple", single particle situations, we can mix in other particles and the move to real physics events.

Ultimately, we want to try and quantify how well we can identify muons with ePIC and hopefully end up with a generic, muon finding algorithm. We can quanitfy the detection in a few ways:

- Efficiency
    - What fraction of muons passing through our system do we detect
- Purity
    - Using our muon finding algorithm, what fraction of our "muons" are actually misidentified pions (or other particles)
- Fake rate
    - At what rate do we falsely identify non-muon particles as muons?
 
We can do this for our particle gun case and then for specific physics channels too.

# Setup

We will need to either download (or stream) some event files in the following locations:

- /volatile/eic/gbxalex/MuonID/Reco/
    - Particle gun files 
- /volatile/eic/gbxalex/September2025_JPsi_RecoOut/
    - Vector meson production files
- /volatile/eic/sjdkay/EpIC_TCS_mumu_18x275/Reco/
    - TCS files
 
We need to run some initial setup commands first. Execute the cells below.

## Setup Code Blocks

### Optional Cell - Run First if using Google Collab

In [ ]:
!pip uninstall --yes cmake ## to work around an xrootd compilation issue on virtual machine
!pip install uproot awkward XRootD numpy pandas matplotlib scipy vector

### Non-Optional - Always Run!

In [4]:
#Import some packages we'll need, specifically, uproot
import uproot as up
import os
import awkward as ak
import numpy as np
import pandas as pd
import scipy
import matplotlib as mpl
import matplotlib.ticker as ticker
import matplotlib.cm as cm
import matplotlib.pylab as plt
import vector
from XRootD import client
from scipy import stats
from matplotlib import pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib import colors as colours

In [7]:
plt.rcParams['figure.figsize'] = [8.0, 6.0]
plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['xaxis.labellocation'] = 'right'
plt.rcParams['yaxis.labellocation'] = 'top'
SMALL_SIZE = 10
MEDIUM_SIZE = 14
BIGGER_SIZE = 20
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title]
deg2rad = np.pi/180.0

In [8]:
# See - https://pythonhosted.org/xrootdfs/ - For some details on available commands
# Create XRootD client
eic_server = 'root://dtn-eic.jlab.org/'
fs = client.FileSystem(eic_server)
GunPath = "/volatile/eic/gbxalex/MuonID/Recon/"
VMPath = "/volatile/eic/gbxalex/September2025_JPsi_RecoOut/"
TCSPath = "/volatile/eic/sjdkay/EpIC_TCS_mumu_18x275/Reco/"

## Testing Paths

We can check each path via:

In [13]:
status, files = fs.dirlist(GunPath)
for entry in files:
    print(entry.name)

proton_20GeV_reconOut.root
kaonMinus_20GeV_reconOut.root
piPlus_20GeV_reconOut.root
MuPiKep_Mixed_RecoOut.root
muMinus_20GeV_reconOut.root
piMinus_20GeV_reconOut.root
electron_20GeV_reconOut.root
kaonPlus_20GeV_reconOut.root
muPlus_20GeV_reconOut.root


We see a list of individual roout files. Each is a single particle type except for "MuPiKep_Mixed_RecoOut.root" - which is a mix of all types.

For vector meson production we have:

In [14]:
status, files = fs.dirlist(VMPath)
for entry in files:
    print(entry.name)

10x250ep
10x130ep


Two beam energy combinations, let's check 10x130ep:

In [15]:
status, files = fs.dirlist(VMPath+"/10x130ep")
for entry in files:
    print(entry.name)

reconOut
logs
simOut


The files in reconOut are what we will need later. Finally, for TCS:

In [16]:
status, files = fs.dirlist(TCSPath)
for entry in files:
    print(entry.name)

TCS_events_mumu_plus_68_recon.root
TCS_events_mumu_plus_98_recon.root
TCS_events_mumu_plus_52_recon.root
TCS_events_mumu_plus_21_recon.root
TCS_events_mumu_plus_44_recon.root
TCS_events_mumu_plus_37_recon.root
TCS_events_mumu_plus_7_recon.root
TCS_events_mumu_plus_59_recon.root
TCS_events_mumu_plus_63_recon.root
TCS_events_mumu_plus_10_recon.root
TCS_events_mumu_plus_93_recon.root
TCS_events_mumu_plus_75_recon.root
TCS_events_mumu_plus_85_recon.root
TCS_events_mumu_plus_48_recon.root
TCS_events_mumu_plus_82_recon.root
TCS_events_mumu_plus_72_recon.root
TCS_events_mumu_plus_94_recon.root
TCS_events_mumu_plus_17_recon.root
TCS_events_mumu_plus_64_recon.root
TCS_events_mumu_plus_89_recon.root
TCS_events_mumu_plus_79_recon.root
TCS_events_mumu_plus_30_recon.root
TCS_events_mumu_plus_43_recon.root
TCS_events_mumu_plus_26_recon.root
TCS_events_mumu_plus_55_recon.root
TCS_events_mumu_plus_0_recon.root
TCS_events_mumu_plus_5_recon.root
TCS_events_mumu_plus_19_recon.root
TCS_events_mumu_plus_46

# Streaming Files

The datasets we have at hand this time are a little large to download and run. Instead, we can stream them. Let's start with a particle gun one. We'll first double check our filesizes here too:

In [17]:
status, files = fs.dirlist(GunPath)
flist=[]
for entry in files:
    flist.append(entry.name) # Add all of our entries to a new list, flist

for entry in sorted(flist):
    fname = GunPath + entry
    print(entry," File Size: ",(((fs.stat(fname))[1]).size)/(1024**2), "MB")

MuPiKep_Mixed_RecoOut.root  File Size:  9173.820505142212 MB
electron_20GeV_reconOut.root  File Size:  551.6948347091675 MB
kaonMinus_20GeV_reconOut.root  File Size:  623.7307968139648 MB
kaonPlus_20GeV_reconOut.root  File Size:  570.8038845062256 MB
muMinus_20GeV_reconOut.root  File Size:  604.6560964584351 MB
muPlus_20GeV_reconOut.root  File Size:  605.3360433578491 MB
piMinus_20GeV_reconOut.root  File Size:  2883.4351301193237 MB
piPlus_20GeV_reconOut.root  File Size:  2769.5769052505493 MB
proton_20GeV_reconOut.root  File Size:  580.133563041687 MB


We'll choese muPlus_20GeV_reconOut.root for now:

In [20]:
muPlusFile = eic_server + GunPath + "muPlus_20GeV_reconOut.root"
file=up.open(muPlusFile)
tree = file['events'] 

We're now good to go and we can start analysing this as we did previously in the May workshop. We can check our tree branches quickly for our MC branches via:

In [21]:
tree.keys(filter_name="MC*", recursive=False)

['MCBeamElectrons_objIdx',
 'MCBeamProtons_objIdx',
 'MCNonScatteredElectronAssociations_objIdx',
 'MCParticles',
 'MCParticlesHeadOnFrameNoBeamFX',
 'MCScatteredElectronAssociations_objIdx',
 'MCScatteredElectrons_objIdx',
 'MCScatteredProtons_objIdx']

# Event Display

To try and understand our events, we will also try the EIC event display with our files. This is a bit of a work in progress, so we'll see how it goes and try to get it working for us!

The event display can be found [here](https://github.com/eic/firebird). There are some instructions on running this and a link on this page to running it in our web browser. We'll need to convert some of our files to the right format.

# Workspace

From here on in, this is just space to work. You can start a fresh notebook if you want or just work in new cells below this, it's up to you! To begin with, we'll look at the MC particle distributions and see how our events are actually distributed. We'll start with the muons, then check other particles. 